In [1]:
import os
import pandas as pd
import regex as re
import numpy as np
import nltk
from nltk.corpus import stopwords

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression 
nltk.download('stopwords')
sw = stopwords.words("english") + ['unto', 'ye', 'shall'] + ['shall', 'unto', 'thou', 'thy', 'thee','thereof']


text = open(os.path.join("/home/atom/Bureau/Aurelien/epitech/seminaire_nlp/pg10.txt")).read()
corpus = re.split("\*{3}",text)

[nltk_data] Downloading package stopwords to /home/atom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
old = corpus[2]
old_verse = re.findall("\d+:\d+(.*?)(?=\d+:\d+|\n+$)",old,re.DOTALL)
old_verse = [re.sub("\n+"," ",v).strip(" ").lower() for v in old_verse]

new = corpus[3]
new_verse = re.findall("\d+:\d+(.*?)(?=\d+:\d+|\n+$)",new,re.DOTALL)
new_verse = [re.sub("\n+"," ",v).strip(" ").lower() for v in new_verse]

df = pd.DataFrame({"testament":["old"]*len(old_verse)+["new"]*len(new_verse),"text":old_verse+new_verse})

In [3]:
verses = df.text.values
verses_clean = [" ".join(re.findall("[^\W\d]+",v)) for v in verses]
verses_clean_nosw = [re.findall("[^\W\d]+",v) for v in verses]
verses_clean_nosw = [" ".join([w for w in verse_tokenized if not w in sw]) for verse_tokenized in verses_clean_nosw]

df.loc[:,"verses_clean"] = verses_clean
df.loc[:,"verses_clean_nosw"] = verses_clean_nosw

In [4]:
df_train, df_test , y_train , y_test = train_test_split(df.drop(["testament"],axis=1),df.testament,test_size=0.2,stratify=df.testament)
ids_train = y_train.index.values
ids_test = y_test.index.values

# Prediction with CountVectorizer

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df.verses_clean.values)
X = pd.DataFrame(data=X.toarray(),columns = vectorizer.get_feature_names_out())
X_train = X.loc[ids_train,:]
X_test = X.loc[ids_test]

In [6]:
clf_swYES_cvMAX = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred_train = clf_swYES_cvMAX.predict(X_train)
y_pred_test = clf_swYES_cvMAX.predict(X_test)

/home/atom/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [591]:
print(accuracy_score(y_train,y_pred_train))
print(accuracy_score(y_test,y_pred_test))

0.9662392990635424
0.927825108503456


In [7]:
print(accuracy_score(y_train,y_pred_train))
print(accuracy_score(y_test,y_pred_test))

0.9667215947912061
0.9337727053528372


In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df.verses_clean_nosw.values)
X = pd.DataFrame(data=X.toarray(),columns = vectorizer.get_feature_names_out())
X_train = X.loc[ids_train,:]
X_test = X.loc[ids_test]

In [9]:
clf_swNO_cvMAX = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred_train = clf_swNO_cvMAX.predict(X_train)
y_pred_test = clf_swNO_cvMAX.predict(X_test)

/home/atom/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
print(accuracy_score(y_train,y_pred_train))
print(accuracy_score(y_test,y_pred_test))

0.9606125155741329
0.9246101912875744


In [11]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(df.verses_clean_nosw.values)
X = pd.DataFrame(data=X.toarray(),columns = vectorizer.get_feature_names_out())
X_train = X.loc[ids_train,:]
X_test = X.loc[ids_test]

In [13]:
clf_swNO_cv5000 = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred_train = clf_swNO_cv5000.predict(X_train)
y_pred_test = clf_swNO_cv5000.predict(X_test)

/home/atom/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
print(accuracy_score(y_train,y_pred_train))
print(accuracy_score(y_test,y_pred_test))

0.9528154013102367
0.9217167657932808


In [18]:
def training(corpus,y_train,y_test,max_features_range) :
    scores = []
    for m in max_features_range : 
        print(m)
        vectorizer = CountVectorizer(max_features=m)
        X = vectorizer.fit_transform(corpus)
        X = pd.DataFrame(data=X.toarray(),columns=vectorizer.get_feature_names_out())
        X_train , X_test = X.loc[ids_train,:] , X.loc[ids_test,:]
        clf = LogisticRegression(random_state=0).fit(X_train,y_train)
        y_pred_test = clf.predict(X_test)
        scores.append(accuracy_score(y_test,y_pred_test))
    return score

In [19]:
scores_swYES = training(df.verses_clean.values,y_train,y_test,max_features_range=[500,1000,5000,10000,20000])

500


/home/atom/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1000


/home/atom/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


5000


...

10000


/home/atom/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


20000


/home/atom/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


NameError: name 'score' is not defined

In [ ]:
scores_swNO = training(df.verses_clean_nosw.values,y_train,y_test,max_features_range=[500,1000,5000,10000,20000])

In [20]:
scores_swYES

NameError: name 'scores_swYES' is not defined